# Part 3 - Model Tuning and Evaluation

In [1]:
# import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso,LassoCV,Ridge,RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
%matplotlib inline
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
#import data
housing_df=pd.read_csv("../datasets/modelling_housing_data.csv",low_memory=False)
housing_df.head()

,id,tranc_yearmonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,sec_sch_affiliation,sec_sch_latitude,sec_sch_longitude
0,88471,2016-05,KALLANG/WHAMPOA,4 ROOM,3B,UPP BOON KENG RD,10 TO 12,90.0,Model A,2006,...,78,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,0,1.317659,103.882504
1,122598,2012-07,BISHAN,5 ROOM,153,BISHAN ST 13,07 TO 09,130.0,Improved,1987,...,45,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892
2,170897,2013-07,BUKIT BATOK,EXECUTIVE,289B,BT BATOK ST 25,13 TO 15,144.0,Apartment,1997,...,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013
3,86070,2012-04,BISHAN,4 ROOM,232,BISHAN ST 22,01 TO 05,103.0,Model A,1992,...,20,1,1.354789,103.844934,389.515528,Catholic High School,253,1,1.354789,103.844934
4,153632,2017-12,YISHUN,4 ROOM,876,YISHUN ST 81,01 TO 03,83.0,Simplified,1987,...,74,0,1.416280,103.838798,312.025435,Orchid Park Secondary School,208,0,1.414888,103.838335


In [3]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56220 entries, 0 to 56219
Data columns (total 77 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         56220 non-null  int64  
 1   tranc_yearmonth            56220 non-null  object 
 2   town                       56220 non-null  object 
 3   flat_type                  56220 non-null  object 
 4   block                      56220 non-null  object 
 5   street_name                56220 non-null  object 
 6   storey_range               56220 non-null  object 
 7   floor_area_sqm             56220 non-null  float64
 8   flat_model                 56220 non-null  object 
 9   lease_commence_date        56220 non-null  int64  
 10  resale_price               56220 non-null  float64
 11  tranc_year                 56220 non-null  int64  
 12  tranc_month                56220 non-null  int64  
 13  mid_storey                 56220 non-null  int

In [4]:
housing_df['id']=housing_df['id'].map(lambda x: str(x))

In [5]:
housing_df.describe()

,floor_area_sqm,lease_commence_date,resale_price,tranc_year,tranc_month,mid_storey,lower,upper,mid,floor_area_sqft,...,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,cutoff_point,sec_sch_affiliation,sec_sch_latitude,sec_sch_longitude
count,56220.000000,56220.000000,5.622000e+04,56220.000000,56220.000000,56220.000000,56220.000000,56220.000000,56220.000000,56220.000000,...,56220.000000,56220.000000,56220.000000,56220.000000,56220.000000,56220.000000,56220.000000,56220.000000,56220.000000,56220.000000
mean,97.403479,1992.486233,4.505056e+05,2016.494753,6.575382,8.305016,7.268783,9.341249,8.305016,1048.451050,...,395.111027,55.392921,0.114781,1.366271,103.839338,509.381286,210.179100,0.031732,1.366004,103.839558
std,24.440079,12.141430,1.440083e+05,2.753986,3.336262,5.543214,5.550374,5.542348,5.543214,263.073008,...,234.113190,17.877585,0.318761,0.042699,0.072729,308.438194,20.019264,0.175289,0.042613,0.072718
min,31.000000,1966.000000,1.650000e+05,2012.000000,1.000000,2.000000,1.000000,3.000000,2.000000,333.684000,...,45.668324,20.000000,0.000000,1.274962,103.687724,38.913475,188.000000,0.000000,1.276029,103.687207
25%,76.000000,1984.000000,3.480000e+05,2014.000000,4.000000,5.000000,4.000000,6.000000,5.000000,818.064000,...,227.366303,44.000000,0.000000,1.337289,103.773754,290.740056,188.000000,0.000000,1.337545,103.776008
50%,95.000000,1992.000000,4.250000e+05,2017.000000,7.000000,8.000000,7.000000,9.000000,8.000000,1022.580000,...,348.876691,54.000000,0.000000,1.364639,103.844210,449.136867,208.000000,0.000000,1.364334,103.842719
75%,113.000000,2001.000000,5.200000e+05,2019.000000,9.000000,11.000000,10.000000,12.000000,11.000000,1216.332000,...,502.179385,67.000000,0.000000,1.394530,103.898773,644.453973,224.000000,0.000000,1.391781,103.899872
max,280.000000,2019.000000,1.248000e+06,2021.000000,12.000000,50.000000,49.000000,51.000000,50.000000,3013.920000,...,3305.841039,110.000000,1.000000,1.456667,103.962919,3638.977233,260.000000,1.000000,1.453280,103.961105


### Feature Selection

The following features will be selected for modelling resale price based on our EDA, as summarised in below table.


|   Variable Type   | Model Feature |
| ----------------------| ------------- |
| Numerical | hdb_age |
| Numerical | mid |
| Numerical | max_floor_lvl  |
| Numerical | floor_area_sqft |
| Numerical | bus_interchange |
| Numerical | mrt_interchange |
| Numerical |mall_nearest_distance|
| Numerical |mrt_nearest_distance| 
| Numerical |hawker_nearest_distance| 
| Numerical |pri_sch_nearest_distance| 
| Numerical |sec_sch_nearest_dist| 
| Categorical | planning_area|
| Categorical | full_flat_type|

As planning area and full flat type are categorical variables, one-hot encoding will be applied to convert them into numerical variables for modelling purposes.

### Baseline Model

We establish the baseline model using a dummy regressor that predicts the resale price as the median resale price of the training dataset.

In [6]:
X = housing_df[['hdb_age','mid','max_floor_lvl','floor_area_sqft','pri_sch_nearest_distance','sec_sch_nearest_dist','hawker_nearest_distance','mrt_nearest_distance',
                'bus_interchange','mrt_interchange','mall_nearest_distance','planning_area','full_flat_type']]
X = pd.get_dummies(X, columns=['planning_area','full_flat_type'],drop_first=True)
y = housing_df['resale_price']

In [7]:
X.head()

,hdb_age,mid,max_floor_lvl,floor_area_sqft,pri_sch_nearest_distance,sec_sch_nearest_dist,hawker_nearest_distance,mrt_nearest_distance,bus_interchange,mrt_interchange,...,full_flat_type_5 ROOM Premium Apartment,full_flat_type_5 ROOM Premium Apartment Loft,full_flat_type_5 ROOM Standard,full_flat_type_5 ROOM Type S2,full_flat_type_EXECUTIVE Adjoined flat,full_flat_type_EXECUTIVE Apartment,full_flat_type_EXECUTIVE Maisonette,full_flat_type_EXECUTIVE Premium Apartment,full_flat_type_EXECUTIVE Premium Maisonette,full_flat_type_MULTI-GENERATION Multi Generation
0,15,11,25,968.760,1138.633422,1138.633422,154.753357,330.083069,0,0,...,0,0,0,0,0,0,0,0,0,0
1,34,8,9,1399.320,415.607357,447.894399,640.151925,903.659703,1,1,...,0,0,0,0,0,0,0,0,0,0
2,24,14,16,1550.016,498.849039,180.074558,1762.082341,1334.251197,1,0,...,0,0,0,0,0,1,0,0,0,0
3,29,3,11,1108.692,389.515528,389.515528,726.215262,907.453484,1,1,...,0,0,0,0,0,0,0,0,0,0
4,34,2,4,893.412,401.200584,312.025435,1540.151439,412.343032,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=30)
base_model = DummyRegressor(strategy='median')
base_model.fit(X_train, y_train)
y_pred=base_model.predict(X_test)
y_pred_train=base_model.predict(X_train)
baseline_RMSE=np.sqrt(metrics.mean_squared_error(y_test,y_pred))
baseline_cross_val_score=cross_val_score(base_model,X_train,y_train,cv=5).mean()
baseline_train_R2_score=r2_score(y_train,y_pred_train)
baseline_test_R2_score=r2_score(y_test,y_pred)

print(f"Baseline RMSE: {baseline_RMSE}")
print(f"Baseline 5-fold validation score: {baseline_cross_val_score}")
print(f"Train R2 score: {baseline_train_R2_score}")
print(f"Test R2 score: {baseline_test_R2_score}")

Baseline RMSE: 144537.63285165015
Baseline 5-fold validation score: -0.03427455111212985
Train R2 score: -0.03454429132781289
Test R2 score: -0.03189280316881682


### Linear Regression

In [9]:
X = housing_df[['hdb_age','mid','max_floor_lvl','floor_area_sqft','pri_sch_nearest_distance','sec_sch_nearest_dist','hawker_nearest_distance','mrt_nearest_distance',
                'bus_interchange','mrt_interchange','mall_nearest_distance','planning_area','full_flat_type']]
X = pd.get_dummies(X, columns=['planning_area','full_flat_type'],drop_first=True)
y = housing_df['resale_price']

In [10]:
#linear regression
lr = LinearRegression()
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=30)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
lr_RMSE=np.sqrt(metrics.mean_squared_error(y_test,y_pred))
lr_cross_val_score=cross_val_score(lr,X_train,y_train,cv=5).mean()
lr_train_R2_score=r2_score(lr.predict(X_train), y_train)
lr_test_R2_score=r2_score(lr.predict(X_test), y_test)
print(f"Linear regression RMSE: {lr_RMSE}")
print(f"Linear regression 5-fold validation score: {lr_cross_val_score}")
print(f"Linear regression train R2 score: {lr_train_R2_score}")
print(f"Linear regression test R2 score: {lr_test_R2_score}")

Linear regression RMSE: 50395.48955550961
Linear regression 5-fold validation score: 0.8727709293936696
Linear regression train R2 score: 0.8551978008546991
Linear regression test R2 score: 0.8578618865141349


In [11]:
lr.intercept_

207300.52039467118

In [12]:
lr.coef_

array([-3.44487155e+03,  3.31542156e+03,  2.87638651e+03,  3.06014294e+02,
        1.33865157e+01,  1.03171907e+01, -1.85959810e+01, -5.87158864e+01,
        6.47576762e+03,  3.60169300e+03, -1.72091696e+01, -1.16557226e+04,
        9.62704625e+04, -4.21462916e+04,  8.64030505e+04, -1.08938925e+05,
        1.40650914e+05,  1.61491652e+05, -1.05325781e+05,  2.71922545e+04,
        1.51849065e+05,  1.94916785e+04, -4.73262815e+04, -4.95717544e+04,
       -9.03736509e+04,  3.14494369e+04,  2.10746603e+05,  2.43047094e+04,
        8.41285083e+04, -5.98915605e+04, -3.55461452e+04,  8.06237608e+04,
        1.47544050e+04, -1.41845924e+05, -7.35149751e+04,  3.56421570e+04,
       -2.57132454e+04,  1.83486472e+05,  3.29834706e+04, -6.50518279e+04,
       -1.35596468e+05, -7.75905537e+04, -2.83783448e+04, -1.41385391e+03,
       -1.20173646e+04,  1.93911830e+04, -1.67035742e+04,  4.66490880e+04,
        8.33262885e+03,  3.40190994e+04,  3.24275758e+04,  1.91682569e+04,
        4.46584626e+04,  

### Lasso Regression

In [13]:
X = housing_df[['hdb_age','mid','max_floor_lvl','floor_area_sqft','pri_sch_nearest_distance','sec_sch_nearest_dist','hawker_nearest_distance','mrt_nearest_distance',
                'bus_interchange','mrt_interchange','mall_nearest_distance','planning_area','full_flat_type']]
X = pd.get_dummies(X, columns=['planning_area','full_flat_type'],drop_first=True)
y = housing_df['resale_price']

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=30)

In [14]:
alphas = np.logspace(-4, 4, 2000)
LS = LassoCV(alphas = alphas ,cv=5, random_state=30).fit(X_train, y_train)
y_pred_with_lasso = LS.predict(X_test)

lasso_RMSE=np.sqrt(metrics.mean_squared_error(y_test,y_pred_with_lasso))
lasso_train_R2_score=r2_score(LS.predict(X_train), y_train)
lasso_test_R2_score=r2_score(LS.predict(X_test), y_test)
print(f"alpha: {LS.alpha_}")
print(f"Lasso RMSE: {lasso_RMSE}")
print(f"Lasso regression train R2 score: {lasso_train_R2_score}")
print(f"Lasso regression test R2 score: {lasso_test_R2_score}")

alpha: 0.0001
Lasso RMSE: 50400.152892111924
Lasso regression train R2 score: 0.8551793483624074
Lasso regression test R2 score: 0.85783064747522


In [15]:
LS.coef_

array([-3.45792538e+03,  3.31517185e+03,  2.88389854e+03,  3.08595546e+02,
        1.33653466e+01,  1.03649024e+01, -1.86002146e+01, -5.87916631e+01,
        6.47459962e+03,  3.60537663e+03, -1.72478859e+01, -1.16590340e+04,
        9.62109527e+04, -4.21460824e+04,  8.60397203e+04, -1.08945143e+05,
        1.40556631e+05,  1.61614813e+05, -1.05395109e+05,  2.72048962e+04,
        1.51814078e+05,  1.94703075e+04, -4.73366028e+04, -4.95980019e+04,
       -9.03400701e+04,  3.13589368e+04,  2.10738151e+05,  2.43243448e+04,
        8.40101766e+04, -5.99914642e+04, -3.55080565e+04,  8.05219127e+04,
        1.46523857e+04, -1.41786114e+05, -7.34628844e+04,  3.56379600e+04,
       -2.57228785e+04,  1.83321559e+05,  3.29227049e+04, -6.51857731e+04,
       -1.35684380e+05, -7.75863524e+04, -5.34098283e+04, -2.59210476e+04,
       -3.70657955e+04, -5.78764875e+03, -4.10178309e+04,  2.09974312e+04,
       -1.66865731e+04,  8.54126686e+03,  7.19096917e+03, -6.49987757e+03,
        1.95126027e+04, -

In [16]:
LS.intercept_

231101.5882953825

### Ridge Regression

In [17]:
X = housing_df[['hdb_age','mid','max_floor_lvl','floor_area_sqft','pri_sch_nearest_distance','sec_sch_nearest_dist','hawker_nearest_distance','mrt_nearest_distance',
                'bus_interchange','mrt_interchange','mall_nearest_distance','planning_area','full_flat_type']]
X = pd.get_dummies(X, columns=['planning_area','full_flat_type'],drop_first=True)
y = housing_df['resale_price']
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=30)

In [18]:
alphas = np.logspace(-4, 4, 2000)
ridge = RidgeCV(alphas = alphas ,cv=5).fit(X_train, y_train)
y_pred_with_ridge = ridge.predict(X_test)

ridge_RMSE=np.sqrt(metrics.mean_squared_error(y_test,y_pred_with_ridge))
ridge_train_R2_score=r2_score(ridge.predict(X_train), y_train)
ridge_test_R2_score=r2_score(ridge.predict(X_test), y_test)
print(f"Ridge regression alpha: {ridge.alpha_}")
print(f"Ridge regression RMSE: {ridge_RMSE}")
print(f"Ridge regression train R2 score: {ridge_train_R2_score}")
print(f"Ridge regression test R2 score: {ridge_test_R2_score}")

Ridge regression alpha: 0.01928157789796517
Ridge regression RMSE: 50396.06002052935
Ridge regression train R2 score: 0.8551890822561222
Ridge regression test R2 score: 0.8578527256709573


In [19]:
ridge.coef_

array([-3.44807425e+03,  3.31540920e+03,  2.87818626e+03,  3.06729035e+02,
        1.33828533e+01,  1.03294014e+01, -1.86013971e+01, -5.87313459e+01,
        6.47393556e+03,  3.60225068e+03, -1.72112387e+01, -1.16617657e+04,
        9.62499428e+04, -4.21438933e+04,  8.63330524e+04, -1.08935952e+05,
        1.40592470e+05,  1.61168502e+05, -1.05329428e+05,  2.71947203e+04,
        1.51546282e+05,  1.94849627e+04, -4.73257753e+04, -4.95836325e+04,
       -9.03623977e+04,  3.14240750e+04,  2.10718268e+05,  2.43266333e+04,
        8.41433493e+04, -5.99184118e+04, -3.55138164e+04,  8.06109612e+04,
        1.47296234e+04, -1.41815836e+05, -7.34891827e+04,  3.56387465e+04,
       -2.57167311e+04,  1.82551067e+05,  3.29659376e+04, -6.46565735e+04,
       -1.35619047e+05, -7.75888376e+04, -3.24130340e+04, -5.63361342e+03,
       -1.63699826e+04,  1.49839786e+04, -2.08775775e+04,  4.21200512e+04,
        3.97000360e+03,  2.95391520e+04,  2.80093899e+04,  1.46395060e+04,
        4.02677103e+04,  

In [20]:
ridge.intercept_

211304.7908395706

No significant improvement was noted in terms of RMSE or R2 score when Lasso or Ridge regularisation was applied. We will rely on existing linear regression model to further understand our coefficients, since there was a significant improvement observed relative to the baseline score.

In [21]:
X = housing_df[['hdb_age','mid','max_floor_lvl','floor_area_sqft','pri_sch_nearest_distance','sec_sch_nearest_dist','hawker_nearest_distance','mrt_nearest_distance',
                'bus_interchange','mrt_interchange','mall_nearest_distance','planning_area','full_flat_type']]
X = pd.get_dummies(X, columns=['planning_area','full_flat_type'],drop_first=True)
y = housing_df['resale_price']

In [22]:
lr = LinearRegression()
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=30)
lr.fit(X_train,y_train)

LinearRegression()

In [23]:
coefficients={'Feature':[x for x in X_train.columns],
              'Coefficient Value':[x for x in lr.coef_]}

coefficients['Category'] = ['full_flat_type' if 'full_flat_type' in var 
                          else 'planning_area' if 'planning_area' in var
                          else 'mrt/bus' if ('mrt' in var) | ('bus' in var)
                          else 'hawker' if 'hawker' in var
                          else 'storey' if (var == 'mid') | ('storey' in var) | ('lvl' in var)
                          else 'mall' if 'mall' in var
                          else 'time' if 'tranc' in var
                          else 'sch' if 'sch' in var
                          else var for var in X_train.columns]

In [24]:
coeff_table=pd.DataFrame(coefficients)
coeff_table.to_csv("../datasets/model_coefficients.csv",index=False)